In [3]:
# Setup

import os
import re
import pandas as pd
from dotenv import load_dotenv

# .env laden
load_dotenv()

# Pfade aus .env (Fallbacks)
LV_PATH = os.getenv("LV_PATH", "data/lv.csv")
LV_OUT_PATH = os.getenv("LV_OUT_PATH", "data/lv_clean.csv")
NROWS = int(os.getenv("LV_NROWS", "0")) or None

print(f"Lese LV aus: {LV_PATH}")
df = pd.read_csv(LV_PATH, nrows=NROWS)

# Minimalprüfungen
required_cols = {"id", "oz_bez"}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Fehlende Spalten: {missing}")

# Python
TRIGGER_PATTERNS = [
    r"\bwie\s+(?:oben|vor|zuvor|zuletzt)\b",
    r"\bw(?:ie)?\s*o\b",            # Abkürzung "w.o."
    r"\bdito\b",
    r"\bentspr\.\s*oben\b",         # entspricht oben
    r"\bgleich\s+wie\b",
    r"\bwie\s+beschr(?:ieben|.)\b",
    r"\bwie\s+pos(?:ition)?\s*\d+\b",  # Bezug auf Positionen
    r"\bwie\s+z(?:ur|u)\s+pos\b",
]
import re
trigger_re = re.compile("|".join(f"(?:{p})" for p in TRIGGER_PATTERNS), flags=re.IGNORECASE)
def has_trigger(text: str) -> bool:
    return bool(trigger_re.search(str(text)))

print("Setup ok.")

Lese LV aus: data/lv.csv


/var/folders/pn/v_lhcbpn6_3_fd89lgk2kz_00000gn/T/ipykernel_37605/318466758.py:17: DtypeWarning: Columns (7,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(LV_PATH, nrows=NROWS)


Setup ok.


In [4]:
# 1) Trigger-Vorschau: Kennzahlen + Beispiele

import pandas as pd
import re

# Sicherstellen, dass df, has_trigger existieren (aus voriger Zelle)
assert isinstance(df, pd.DataFrame), "df fehlt – bitte Einlesezelle vorher ausführen."

df_preview = df.copy()


# Normalisierung für Suche
def norm_txt(x):
    return str(x).lower()


df_preview["_oz_norm"] = df_preview["oz_bez"].map(norm_txt)

# Treffer markieren
df_preview["_has_trigger"] = df_preview["_oz_norm"].apply(has_trigger)

# Kennzahlen
total = len(df_preview)
hits = int(df_preview["_has_trigger"].sum())
rate = hits / total if total else 0.0

print("Trigger-Analyse")
print(f"- Zeilen gesamt: {total}")
print(f"- Zeilen mit Trigger: {hits} ({rate:.2%})")

# N-Gramme / Häufigkeit je Triggerwort
from collections import Counter

word_counts = Counter()
for t in trigger_words:
    t_l = t.lower()
    cnt = int(df_preview["_oz_norm"].str.contains(re.escape(t_l), na=False).sum())
    word_counts[t] = cnt

print("\nHäufigkeit je Trigger:")
for k, v in word_counts.most_popular() if hasattr(word_counts, "most_popular") else word_counts.most_common():
    print(f"- {k}: {v}")

# Beispiele anzeigen
print("\nBeispiele (erste 10 Treffer):")
examples = df_preview.loc[df_preview["_has_trigger"], ["id", "oz_bez"]].head(10)
display(examples)

# Verdachtsfälle: sehr kurze Texte mit Trigger (könnten “falsche” Matches sein)
print("\nVerdachtsfälle (kurze Texte mit Trigger, erste 10):")
short_susp = df_preview.loc[df_preview["_has_trigger"] & (df_preview["oz_bez"].str.len() < 40), ["id", "oz_bez"]].head(
    10)
display(short_susp)

# Aufräumen
df_preview.drop(columns=["_oz_norm", "_has_trigger"], inplace=True)

Trigger-Analyse
- Zeilen gesamt: 5860441
- Zeilen mit Trigger: 319641 (5.45%)

Häufigkeit je Trigger:
- wie vor: 272788
- wie zuletzt: 35575
- wie zuvor: 26399
- wie oben: 6472
- dito: 1570

Beispiele (erste 10 Treffer):


,id,oz_bez
30,283,"wie vor, jedoch 16 A.\n wie vor, jedoch 16 A.\n\n"
31,284,"wie vor, jedoch 20 A.\n wie vor, jedoch 20 A.\n"
32,285,"wie vor, jedoch 10 A, 3-polig.\n wie vor, jedo..."
34,287,"wie vor, jedoch 20 A, 3-polig.\n wie vor, jedo..."
51,328,"wie vor, jedoch\nGröße 300 x 450 mm.\n wie vor..."
52,329,"wie vor, jedoch\nGröße 300 x 600 mm.\n wie vor..."
75,1389,"wie vor, jedoch ca. 15x 15mm\n wie vor, jedoch..."
76,1390,"wie vor, jedoch ca. 15 x 30mm\n wie vor, jedoc..."
77,1391,"wie vor, jedoch ca 60 x 90mm\n wie vor, jedoch..."
78,1392,wie vor jedoch ca 60 x 110mm mit Trennsteg\n w...



Verdachtsfälle (kurze Texte mit Trigger, erste 10):


,id,oz_bez
3937,11029,Leerrohr Leerrohr wie vor jedoch 25mm\n
3938,11030,Leerrohr Leerrohr wie vor jedoch 32mm.\n
3939,11031,Leerrohr Leerrohr wie vor jedoch 40mm.\n
3941,11033,Leerrohr Leerrohr wie vor jedoch 32mm\n
3942,11034,Leerrohr Leerrohr wie vor jedoch 40mm.\n
3943,11035,Leerrohr Leerrohr wie vor jedoch 50mm.\n
6206,14672,"desgleichen wie vor, jedoch 400 Amp."
6377,14769,"desgleichen wie vor, jedoch DN 25"
6378,14770,"desgleichen wie vor, jedoch DN 32"
6416,14811,"desgleichen wie vor, jedoch 32 A"


In [5]:
# 2) Trockentest (Dry-Run): Zeige, was geändert WÜRDE, ohne zu schreiben

# Index vorbereiten wie in der Reinigungslogik
first_by_id = (
    df.sort_values("id")
    .groupby("id", as_index=True)["oz_bez"]
    .first()
)


def find_base_text(current_id: int):
    search_id = current_id - 1
    while True:
        if search_id not in first_by_id.index:
            return None
        text = first_by_id.loc[search_id]
        if not has_trigger(text):
            return text
        search_id -= 1


# Beispiele der geplanten Änderungen sammeln
planned = []
limit = 20  # Anzahl Beispiele
for i, row in df.iterrows():
    try:
        current_id = int(row["id"])
    except Exception:
        continue
    current_text = row["oz_bez"]
    if has_trigger(current_text):
        base_text = find_base_text(current_id)
        if base_text:
            cleaned = f"{base_text} {current_text}"
            cleaned = cleaned.replace("\\n", " ")
            cleaned = re.sub(r"\\s+", " ", cleaned).strip()
            planned.append({
                "id": current_id,
                "base_text": base_text,
                "current_text": current_text,
                "would_cleaned": cleaned
            })
            if len(planned) >= limit:
                break

print(f"Geplante Änderungen (Beispiele bis {limit}): {len(planned)} gezeigt")
display(pd.DataFrame(planned))

Geplante Änderungen (Beispiele bis 20): 20 gezeigt


,id,base_text,current_text,would_cleaned
0,283,"LEITUNGSSCHUTZSCHALTER\nnach VDE 0641, 400 V W...","wie vor, jedoch 16 A.\n wie vor, jedoch 16 A.\n\n","LEITUNGSSCHUTZSCHALTER\nnach VDE 0641, 400 V W..."
1,284,"LEITUNGSSCHUTZSCHALTER\nnach VDE 0641, 400 V W...","wie vor, jedoch 20 A.\n wie vor, jedoch 20 A.\n","LEITUNGSSCHUTZSCHALTER\nnach VDE 0641, 400 V W..."
2,285,"LEITUNGSSCHUTZSCHALTER\nnach VDE 0641, 400 V W...","wie vor, jedoch 10 A, 3-polig.\n wie vor, jedo...","LEITUNGSSCHUTZSCHALTER\nnach VDE 0641, 400 V W..."
3,287,"LEISTUNGSSCHALTER\nnach VDE 0641, 400 V WS, Ab...","wie vor, jedoch 20 A, 3-polig.\n wie vor, jedo...","LEISTUNGSSCHALTER\nnach VDE 0641, 400 V WS, Ab..."
4,328,"VERTEILERKASTEN\naus Polycarbonat, RAL 7032 mi...","wie vor, jedoch\nGröße 300 x 450 mm.\n wie vor...","VERTEILERKASTEN\naus Polycarbonat, RAL 7032 mi..."
5,329,"VERTEILERKASTEN\naus Polycarbonat, RAL 7032 mi...","wie vor, jedoch\nGröße 300 x 600 mm.\n wie vor...","VERTEILERKASTEN\naus Polycarbonat, RAL 7032 mi..."
6,1389,Leerrohr EN 40\n Leerrohr EN 40\n,"wie vor, jedoch ca. 15x 15mm\n wie vor, jedoch...","Leerrohr EN 40\n Leerrohr EN 40\n wie vor, jed..."
7,1390,Leerrohr EN 40\n Leerrohr EN 40\n,"wie vor, jedoch ca. 15 x 30mm\n wie vor, jedoc...","Leerrohr EN 40\n Leerrohr EN 40\n wie vor, jed..."
8,1391,Leerrohr EN 40\n Leerrohr EN 40\n,"wie vor, jedoch ca 60 x 90mm\n wie vor, jedoch...","Leerrohr EN 40\n Leerrohr EN 40\n wie vor, jed..."
9,1392,Leerrohr EN 40\n Leerrohr EN 40\n,wie vor jedoch ca 60 x 110mm mit Trennsteg\n w...,Leerrohr EN 40\n Leerrohr EN 40\n wie vor jedo...


In [6]:
# Explorative Qualitätschecks (nur Anzeige, keine Änderungen)

import pandas as pd
import numpy as np
import re

assert {"oz_bez", "arnr"}.issubset(df.columns), "Erwarte Spalten oz_bez, arnr."
gruppen_col = next((c for c in df.columns if c.lower().startswith("gruppen") and "bez" in c.lower()), None)

subset_cols = ["oz_bez", "arnr"] + ([gruppen_col] if gruppen_col else [])
dsub = df[subset_cols].copy()

print(f"Arbeite mit Spalten: {subset_cols}")


# 1) Grundlegende Null-/Leere-Werte
def is_blank(s):
    return s.isna() | (s.astype(str).str.strip() == "")


report = {}

report["oz_bez_null"] = int(dsub["oz_bez"].isna().sum())
report["oz_bez_blank"] = int(is_blank(dsub["oz_bez"]).sum())
report["arnr_null"] = int(dsub["arnr"].isna().sum())
report["arnr_blank"] = int(is_blank(dsub["arnr"]).sum())
if gruppen_col:
    report["gruppen_bez_null"] = int(dsub[gruppen_col].isna().sum())
    report["gruppen_bez_blank"] = int(is_blank(dsub[gruppen_col]).sum())

print("Null/Blank Übersicht:", report)

# 2) Duplikate und Near-Duplicates
dup_exact = dsub.duplicated(subset=["oz_bez", "arnr"], keep=False).sum()
print(f"Exakte Duplikate (oz_bez+arnr): {dup_exact}")

# 3) ARNR-Label-Leakage: Eine oz_bez -> viele arnr?
vc_multi_arnr = (
    dsub.groupby("oz_bez")["arnr"].nunique(dropna=True)
    .sort_values(ascending=False)
)
ambiguous = vc_multi_arnr[vc_multi_arnr > 1].head(20)
print(f"Ambigue oz_bez (mehrere arnr): {int((vc_multi_arnr > 1).sum())}")
display(ambiguous.to_frame("distinct_arnr").head(20))


# 4) Token-/Text-Artefakte: Wiederholungen, Steuerzeichen, Mehrfach-Whitespace
def find_issues(series, limit=20):
    ser = series.astype(str)
    out = {}

    # Wiederholte Phrasen (z.B. “wie vor, ... wie vor, ...”)
    rep_mask = ser.str.contains(r"(.+)\s+\1", flags=re.IGNORECASE)
    out["repeated_phrase_examples"] = ser[rep_mask].head(limit)

    # Steuerzeichen/übermäßige Newlines
    ctrl_mask = ser.str.contains(r"[\r\t]|\\n|(\n.*\n.*\n)")  # literal \n oder viele echte \n
    out["control_char_examples"] = ser[ctrl_mask].head(limit)

    # Mehrfach-Whitespace
    multi_ws = ser.str.contains(r"\s{2,}")
    out["multi_whitespace_examples"] = ser[multi_ws].head(limit)

    # Sehr kurze Texte (evtl. Referenz/Positionszeilen)
    short = ser.str.len() < 15
    out["very_short_examples"] = ser[short].head(limit)

    # Sehr lange Texte (evtl. Fließtext, unpassend fürs Training)
    long = ser.str.len() > 500
    out["very_long_examples"] = ser[long].head(limit)

    return out


issues = find_issues(dsub["oz_bez"])
print("Wiederholte Phrasen (Beispiele):")
display(issues["repeated_phrase_examples"].to_frame("oz_bez"))
print("Steuerzeichen/übermäßige Newlines (Beispiele):")
display(issues["control_char_examples"].to_frame("oz_bez"))
print("Mehrfach-Whitespace (Beispiele):")
display(issues["multi_whitespace_examples"].to_frame("oz_bez"))
print("Sehr kurz (Beispiele):")
display(issues["very_short_examples"].to_frame("oz_bez"))
print("Sehr lang (Beispiele):")
display(issues["very_long_examples"].to_frame("oz_bez"))

# 5) Zahlen-/Maßeinheiten-Konsistenz (oft wichtig für Elektro/BAU)
UNIT_PAT = r"(?:mm|cm|m|m²|m3|st|stk|stck|a|v|kv|w|kw|kva|a\-|da|dn|ø|ohm|nm|mm²|qmm|kg|g)\b"
NUM_UNIT = dsub["oz_bez"].astype(str).str.contains(r"\d+\s*(?:x\s*)?\d*\s*" + UNIT_PAT, flags=re.IGNORECASE)
print(f"Texte mit Zahlen/Einheiten: {int(NUM_UNIT.sum())} ({NUM_UNIT.mean():.2%})")
display(dsub.loc[NUM_UNIT].head(10))

# 6) Verdächtige Referenzen jenseits 'wie vor' (weitere Phrasen)
REF_PATTERNS = [
    r"\bw\.?o\.?\b",  # w.o.
    r"\bdito\b",
    r"\bentspr\.\b",
    r"\bgleich\s+wie\b",
    r"\bsiehe\s+(?:oben|pos|position)\b",
    r"\bgemäß\s+pos\b",
    r"\bwie\s+beschr\w*\b",
    r"\bdesgleichen\b",
]
ref_re = re.compile("|".join(f"(?:{p})" for p in REF_PATTERNS), re.IGNORECASE)
ref_mask = dsub["oz_bez"].astype(str).str.contains(ref_re)
print(f"Weitere Referenz-Phrasen: {int(ref_mask.sum())} ({ref_mask.mean():.2%})")
display(dsub.loc[ref_mask].head(20))

# 7) Gruppenbezeichnung-Qualität (falls vorhanden)
if gruppen_col:
    grp_blank = is_blank(dsub[gruppen_col])
    print(f"Leere gruppen_bez: {int(grp_blank.sum())}")
    # Gruppennamen mit starkem Rauschen (sehr generisch)
    generic_grp = dsub[gruppen_col].astype(str).str.fullmatch(r"\s*(?:allgemein|divers|sonstiges|ohne)\s*", case=False,
                                                              na=False)
    print(f"Generische gruppen_bez: {int(generic_grp.sum())}")
    display(dsub.loc[generic_grp, [gruppen_col]].drop_duplicates().head(20))

# 8) ARNR-Qualität: nicht-numerisch / Formatabweichungen
arnr_str = dsub["arnr"].astype(str).str.strip()
non_numeric = ~arnr_str.str.fullmatch(r"[0-9 .-]+")
print(f"ARNR nicht-numerisch/mit Buchstaben: {int(non_numeric.sum())}")
display(dsub.loc[non_numeric, ["arnr"]].drop_duplicates().head(20))

# 9) Label-Imbalance: Top/Long Tail
top_arnr = dsub["arnr"].value_counts(dropna=False).head(20)
print("Top 20 ARNR (Häufigkeit):")
display(top_arnr)

tail_ratio = (dsub["arnr"].value_counts() == 1).mean()
print(f"Anteil Singleton-ARNR (nur 1 Vorkommen): {tail_ratio:.2%}")

# 10) Feldtrennung in oz_bez: häufige Trennzeichen (für spätere Normalisierung)
sep_counts = {
    "slash": dsub["oz_bez"].astype(str).str.contains(r"/").mean(),
    "comma": dsub["oz_bez"].astype(str).str.contains(r",").mean(),
    "semicolon": dsub["oz_bez"].astype(str).str.contains(r";").mean(),
    "dash": dsub["oz_bez"].astype(str).str.contains(r"\s-\s| - ").mean(),
    "paren": dsub["oz_bez"].astype(str).str.contains(r"[\(\)]").mean(),
}
print("Trennzeichen-Anteile:", {k: f"{v:.2%}" for k, v in sep_counts.items()})

Arbeite mit Spalten: ['oz_bez', 'arnr', 'gruppen_bez']
Null/Blank Übersicht: {'oz_bez_null': 0, 'oz_bez_blank': 967, 'arnr_null': 0, 'arnr_blank': 0, 'gruppen_bez_null': 92, 'gruppen_bez_blank': 5622}
Exakte Duplikate (oz_bez+arnr): 2835785
Ambigue oz_bez (mehrere arnr): 669227


,distinct_arnr
oz_bez,
,758
"Zusätzlicher Windwächter\n Zusätzlicher Windwächter für fassadenbezogene Erfassung\nder Windstärke, zur Aufschaltung auf vg. Zentrale\n",365
NYY-J 5 x 25 mm²\n NYY-J 5 x 25 mm²\n,260
NYY-J 5 x 6 mm²\n NYY-J 5 x 6 mm²\n,247
Kontroll-Schalter Aufputz/Wasserdicht\n Kontroll-Schalter Aufputz/Wasserdicht\n,225
Mehrpreis Kaberlrinne schwarz\n Mehrpreis für Lieferung Kabelrinne bis Breite 300mm in\nder Farbe schwarz.\n,177
"Sicherungslasttrennschalter D02 3x50A\n Sicherungslasttrennschalter D02 3x50A\n\nSicherungslasttrennschalter DIN EN 60947-3 (VDE 0660-107), als Reiheneinbaugerät, Maße DIN 43880, Bemessungsisolationsspannung 440 V AC, einschl. Passeinsatz, bedingter Bemessungskurzschlussstrom 50 kA, fingersicher DIN EN 50274 (VDE 0660-514), zur Montage auf Tragschiene DIN EN 60715, Baugröße D 02, Bemessungsbetriebsspannung 400 V AC, Gebrauchskategorie AC 22, 3-polig, mit Sicherungseinsatz, Bemessungsstrom 50 A.\n",174
"Leitungsschutzschalter B 10A oder B 16A\n Leitungsschutzschalter B 10A oder B 16A\n\nLeitungsschutzschalter wie vor beschrieben, jedoch\nBemessungsstrom 10 A oder 16A\n",150
Potentialausgleichsverbindungen\n Potentialausgleichsverbindungen\nfür Metall- Ständerwände\nPauschal\n\n\n,134


/var/folders/pn/v_lhcbpn6_3_fd89lgk2kz_00000gn/T/ipykernel_37605/4211064042.py:53: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  rep_mask = ser.str.contains(r"(.+)\s+\1", flags=re.IGNORECASE)
/var/folders/pn/v_lhcbpn6_3_fd89lgk2kz_00000gn/T/ipykernel_37605/4211064042.py:57: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ctrl_mask = ser.str.contains(r"[\r\t]|\\n|(\n.*\n.*\n)")  # literal \n oder viele echte \n


Wiederholte Phrasen (Beispiele):


,oz_bez
4,Kleinverteiler\nals typgeprüfte Niederspannung...
5,"ZLS 4 Feld 950 x 1050\n\n Zähler-Leerschrank,..."
6,EBF f. ZS 950 mm/A.s.r. 150 mm\n\n Zählereinb...
7,EBF f. ZS 950 mm\n\n Zählereinbaufeld für Nor...
8,Striebel&J Anschlußklemme VE4 ZK81P4\n Striebe...
9,"S-Hauptleitungsschutzschalt.E,3p.50A,S.S\n\n S..."
10,Steuerleitungsklemmen 7 pol.\n\n Steuerleitung...
12,Unterputzverteiler IP30/4reihig 48TE\n\n U...
20,Einheiten-Drehstromverbinder\n\n Einheiten-Dre...
24,"Vert.-Einb. FI-Sch. 4p. SK, 40A/0,03A\n\n V..."


...

ARNR nicht-numerisch/mit Buchstaben: 1151


,arnr
3992,L000852
16550,L000211
20889,L000850
21902,L003391
28082,1000863GC
29631,L004781
106303,1000871GC
114273,4943798GC
154613,L009816
154614,L009790


Top 20 ARNR (Häufigkeit):


arnr
9900102     81060
97045341    57847
1145084     41476
1322478     38354
97045341    32018
1321342     31877
1021310     20058
1021320     19788
1600099     18760
1021510     17742
1322482     16348
1021520     15936
1071125     15685
2150190     15349
1071126     15308
1000070     14226
1323606     13547
4990171     13245
1322715     12786
1323730     11974
Name: count, dtype: int64

Anteil Singleton-ARNR (nur 1 Vorkommen): 37.00%
Trennzeichen-Anteile: {'slash': '34.97%', 'comma': '79.60%', 'semicolon': '2.91%', 'dash': '10.88%', 'paren': '30.72%'}


In [7]:
# Konkrete Bereinigungsvorschläge (noch ohne Persistenz)

# 1) Whitespace- und Steuerzeichen-Normalisierung
def normalize_text(s: pd.Series) -> pd.Series:
    t = s.astype(str)
    t = t.str.replace(r"\r", " ", regex=True)
    t = t.str.replace(r"\\n", " ", regex=True)  # literal "\n" entfernen
    t = t.str.replace(r"\n", " ", regex=True)  # echte Newlines entfernen
    t = t.str.replace(r"\t", " ", regex=True)
    t = t.str.replace(r"\s{2,}", " ", regex=True)
    return t.str.strip()


tmp = dsub.copy()
tmp["oz_bez_norm"] = normalize_text(tmp["oz_bez"])

# 2) Klein-/Großschreibung für robuste Duplikaterkennung (für Training später evtl. original behalten)
tmp["oz_bez_lower"] = tmp["oz_bez_norm"].str.lower()

# 3) Entferne Führungsworte, die rein formal sind (z.B. “Position …”, “wie beschrieben …”) – erst sichtbar machen
FORMAL_PAT = re.compile(r"^(?:pos(?:ition)?\s*\d+[:.)-]?\s*)", re.IGNORECASE)
mask_formal = tmp["oz_bez_norm"].str.contains(FORMAL_PAT)
print(f"Formale Präfixe (Position...): {int(mask_formal.sum())}")
display(tmp.loc[mask_formal, ["oz_bez", "oz_bez_norm"]].head(10))


# 4) Kandidaten für “nur Referenz, kaum Substanz” (für spätere Behandlung)
def weak_content_score(text: str) -> float:
    t = str(text).lower()
    tokens = re.findall(r"[a-zäöüß]+", t)
    if not tokens:
        return 1.0
    stop = {"wie", "und", "oder", "zu", "zur", "zum", "der", "die", "das", "den", "des", "dem", "dass", "entspricht",
            "dito", "siehe", "gemäß", "laut"}
    sw = sum(tok in stop for tok in tokens)
    return sw / max(1, len(tokens))


tmp["weak_score"] = tmp["oz_bez_lower"].map(weak_content_score)
weak_mask = (tmp["weak_score"] > 0.55) & ~tmp["oz_bez_lower"].str.contains(r"\d")
print(f"Weak-Content-Kandidaten: {int(weak_mask.sum())}")
display(tmp.loc[weak_mask, ["oz_bez"]].head(20))

# 5) Ambiguitätenliste (oz_bez -> n arnr)
ambig = (
    tmp.groupby("oz_bez_lower")["arnr"]
    .nunique()
    .reset_index(name="distinct_arnr")
    .query("distinct_arnr > 1")
    .sort_values("distinct_arnr", ascending=False)
)
print(f"Ambiguitäten gesamt: {len(ambig)}")
display(ambig.head(20))

# 6) Near-Duplicate-Clustering-Vorschlag (lightweight, optional)
# Hinweis: das ist nur zur Sichtung, keine Persistenz.
from difflib import SequenceMatcher


def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()


sample = tmp["oz_bez_lower"].dropna().drop_duplicates().sample(min(1000, tmp["oz_bez_lower"].nunique()),
                                                               random_state=42)
sample = sample.tolist()
pairs = []
thresh = 0.9
for i in range(min(300, len(sample))):
    for j in range(i + 1, min(300, len(sample))):
        r = similar(sample[i], sample[j])
        if r >= thresh:
            pairs.append((sample[i], sample[j], r))
print(f"Near-duplicate Paare (Beispiele, threshold {thresh}): {len(pairs)}")
display(pd.DataFrame(pairs, columns=["a", "b", "sim"]).head(20))

Formale Präfixe (Position...): 32


,oz_bez,oz_bez_norm
956341,Pos 28 Runde LED-Hängeleuchte Beratung\n Exklu...,Pos 28 Runde LED-Hängeleuchte Beratung Exklusi...
956342,Pos 28 Runde LED-Hängeleuchte Beratung\n Exklu...,Pos 28 Runde LED-Hängeleuchte Beratung Exklusi...
3327176,Pos 1. Lichtband E-Line Next zu 43 Lplg.\n Lic...,Pos 1. Lichtband E-Line Next zu 43 Lplg. Licht...
4201328,Pos 01.01.01 - Lüftungsklappen-Stellantrieb (S...,Pos 01.01.01 - Lüftungsklappen-Stellantrieb (S...
4201329,Pos 01.01.03 - Lüftungsklappen-Stellantrieb (A...,Pos 01.01.03 - Lüftungsklappen-Stellantrieb (A...
4201330,Pos 01.01.04 - Lüftungsklappen-Stellantrieb (S...,Pos 01.01.04 - Lüftungsklappen-Stellantrieb (S...
4201331,Pos 01.01.07 - Lüftungsklappen-Stellantrieb (A...,Pos 01.01.07 - Lüftungsklappen-Stellantrieb (A...
4201332,Pos 01.01.08 - Hilfsschalter für Lüftungsklapp...,Pos 01.01.08 - Hilfsschalter für Lüftungsklapp...
4201333,Pos 01.01.10 - Luftkanaltemperaturfühler (Pass...,Pos 01.01.10 - Luftkanaltemperaturfühler (Pass...
4201334,Pos 01.01.13 - Differenzdruckschalter (Messber...,Pos 01.01.13 - Differenzdruckschalter (Messber...


Weak-Content-Kandidaten: 974


,oz_bez
439676,
672881,\n
672882,\n
672883,\n
1009857,
1009858,
1009859,
1009860,
1009861,
1009862,


Ambiguitäten gesamt: 600045


,oz_bez_lower,distinct_arnr
0,,761
1571326,zusätzlicher windwächter zusätzlicher windwäch...,367
1097518,nyy-j 5 x 25 mm² nyy-j 5 x 25 mm²,261
1098095,nyy-j 5 x 6 mm² nyy-j 5 x 6 mm²,250
800181,kontroll-schalter aufputz/wasserdicht kontroll...,225
979846,mehrpreis kaberlrinne schwarz mehrpreis für li...,177
1296016,sicherungslasttrennschalter d02 3x50a sicherun...,174
1536899,"wie vor, jedoch 300 mm breit wie vor, jedoch 3...",152
902408,leitungsschutzschalter b 10a oder b 16a leitun...,150
229339,brüstungskanal nennmaß 130/70 mm brüstungskana...,148


Near-duplicate Paare (Beispiele, threshold 0.9): 1


,a,b,sim
0,verbindungsdose kunststoff 80/80mm t 50mm ip44...,verbindungsdose kunststoff grau 80/80mm t 37mm...,0.925466
